In [1]:
# AI Learns the Numbers Second Model

## Initial Setup

In [2]:
# Assuming you've collected the loss and accuracy values during training
results = {
    "train_loss": train_loss_values,
    "train_acc": train_acc_values,
    "test_loss": test_loss_values,
    "test_acc": test_acc_values
}

plot_loss_curves(results)

NameError: name 'train_loss_values' is not defined

In [ ]:
test_acc = 0
model.eval()
with torch.inference_mode():
    for batch, (X_test, y_test) in enumerate(test_dataloader):
        X_test, y_test = X_test.to(device), y_test.to(device)
        test_pred = model(X_test)
        test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

    # Average accuracy
    test_acc /= len(test_dataloader)
print(f"Test Accuracy: {test_acc:.2f}%")

In [ ]:
test_acc = 0
model.eval()
with torch.inference_mode():
    for batch, (X_test, y_test) in enumerate(test_dataloader):
        X_test, y_test = X_test.to(device), y_test.to(device)
        test_pred = model(X_test)
        test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

    # Average accuracy
    test_acc /= len(test_dataloader)
print(f"Test Accuracy: {test_acc:.2f}%")

In [ ]:
model_weights_path = "modelWeights/trained_model.pth"

# Check if the model weights exist, then load them
if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"[INFO] Loaded pre-trained weights from {model_weights_path}")
else:
    print("[INFO] No pre-trained weights found, training from scratch.")

In [ ]:
# Import libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import random
import requests
from pathlib import Path
from tqdm.auto import tqdm

In [ ]:
# Creating and downloading the MNIST dataset
train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

In [ ]:
# Write Device Agnostic Code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## Dataset Characteristics
This is to explore the MNIST dataset to see what type of data we're going to train our model on.

In [ ]:
# Size of Dataset
len(train_data), len(test_data)

In [ ]:
# Dataset Labels
train_data.classes

In [ ]:
# Plot One Image
CHANGE_THIS_VARIABLE_TO_SEE_ANOTHER_NUMBER = 55
image, label = train_data[CHANGE_THIS_VARIABLE_TO_SEE_ANOTHER_NUMBER]
plt.imshow(image.squeeze(), cmap="gray")
plt.title("Number "+str(label))

In [ ]:
# Plot 40 Random Images from the Datset - Re-run Code to see new ones
fig = plt.figure(figsize=(22.5, 9))
rows, cols = 4, 10
for i in range(1, rows*cols+1):
  image, label = train_data[random.randint(0,len(train_data))]
  fig.add_subplot(rows, cols, i)
  plt.imshow(image.squeeze(), cmap="gray")
  plt.title("Number "+str(label))
  plt.axis(False)

In [ ]:
# Make PyTorch DataLoader
BATCH_SIZE = 32

# Dataset Used to Train the Model
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

# Dataset Used to Test the Model
test_dataloader = DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              shuffle=False)
print(train_dataloader.dataset)
print("")
print(test_dataloader.dataset)

In [ ]:
# Data Preparation
# Define the transformation for data normalization
import torchvision
from torchvision import datasets, transforms

# Specify the dataset path clearly
dataset_path = r"C:\Users\pinka\neuralFromScratch\neuralFromScratch\data\MNIST\raw"

# Define transforms for the dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load training and test datasets
trainset = torchvision.datasets.MNIST(root=dataset_path, train=True, download=False, transform=transform)
testset = torchvision.datasets.MNIST(root=dataset_path, train=False, download=False, transform=transform)

# Create data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
# Verify data loading
data_iter = iter(trainloader)
images, labels = next(data_iter)

# Print the shapes of images and labels to confirm loading
print(f"Images batch shape: {images.shape}")
print(f"Labels batch shape: {labels.shape}")

## Creating the Model

In [ ]:
# Make Model 2
# The structure can be seen below (1 Flatten layer, 2 Linear Layers)
class DumbModel(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_units: int,
               output_shape: int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=hidden_units),
        nn.Linear(in_features=hidden_units, out_features=output_shape),
    )
  def forward(self, x):
    return self.layer_stack(x)


In [ ]:
model = DumbModel(input_shape=784,
                      hidden_units=256,
                      output_shape=len(train_data.classes)).to(device)
model

In [ ]:
# Getting helper functions from my friend daniel bourke

if Path("helper_functions.py").is_file():
  print("Python script already downloaded")
else:
  print("Downloading helper_functions.py script")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn
from helper_functions import print_train_time
from timeit import default_timer as timer

# Creating and choosing a loss function and an optimizer
# Change the learning rate to make the network learn faster (and generally less accurate) or slower (and generally more accurate)
LEARNING_RATE = 0.05

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr=LEARNING_RATE,)

In [ ]:
import torch
from torch import nn

# Define the Neural Network class
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()  # Flattens the input from (1, 28, 28) to (1, 28*28)
        self.layer_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),  # Input layer: 28*28 (784), Output layer: 512
            nn.ReLU(),                # Activation function
            nn.Linear(512, 256),      # Hidden layer: 512, Output layer: 256
            nn.ReLU(),                # Activation function
            nn.Linear(256, 10)        # Hidden layer: 256, Output layer: 10 (10 classes)
        )

    def forward(self, x):
        x = self.flatten(x)  # Flatten the input image
        logits = self.layer_stack(x)  # Forward pass through the layers
        return logits

# Instantiate the model
model = NeuralNetwork()

# Move the model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

In [ ]:
# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
import os

# Load pre-trained model weights if available
model_save_path = "modelWeights/trained_model.pth"
if os.path.exists(model_save_path):
    model.load_state_dict(torch.load(model_save_path, map_location=device))
    model.to(device)  # Ensure the model is set to the correct device
    model.eval()  # Set model to evaluation mode to disable dropout, etc.
    print(f"[INFO] Loaded pre-trained weights from {model_save_path}")
else:
    print("[INFO] No pre-trained weights found. Starting from scratch.")

In [ ]:
from datetime import datetime

# Create a log file to save the metrics
log_file = "training_log.txt"
with open(log_file, "a") as f:
    f.write(f"Training started at: {datetime.now()}\n")

# Training Loop with Improved Logging
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_preds = 0
    total_samples = 0

    for batch_idx, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track running loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        # Log batch results
        if batch_idx % 100 == 99:  # Log every 100 mini-batches
            batch_accuracy = correct_preds / total_samples * 100
            print(f"[Epoch {epoch + 1}/{epochs}, Batch {batch_idx + 1}/{len(trainloader)}] "
                  f"Loss: {running_loss / 100:.4f}, Accuracy: {batch_accuracy:.2f}%")

            # Write to log file
            with open(log_file, "a") as f:
                f.write(f"[{datetime.now()}] Epoch: {epoch + 1}, Batch: {batch_idx + 1}, "
                        f"Loss: {running_loss / 100:.4f}, Accuracy: {batch_accuracy:.2f}%\n")
            
            running_loss = 0.0
            correct_preds = 0
            total_samples = 0

    # End of epoch logging
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_loss += loss_fn(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            test_acc += (predicted == labels).sum().item()

    test_loss /= len(testloader)
    test_acc = (test_acc / len(testset)) * 100

    print(f"[Epoch {epoch + 1}] Train Loss: {running_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")
    with open(log_file, "a") as f:
        f.write(f"[{datetime.now()}] Epoch: {epoch + 1}, Train Loss: {running_loss:.4f}, "
                f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%\n")

with open(log_file, "a") as f:
    f.write(f"Training ended at: {datetime.now()}\n")

## Training the Model

In [ ]:
from time import time

# Define training parameters
epochs = 5
train_loss_values, train_acc_values, test_loss_values, test_acc_values = [], [], [], []

# Start training loop
for epoch in range(epochs):
    start_time = time()
    model.train()
    running_loss, running_accuracy = 0.0, 0.0

    # Training phase
    for batch_idx, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item()
        acc = accuracy_fn(y_true=labels, y_pred=outputs.argmax(dim=1))
        running_accuracy += acc

        # Log progress every 100 batches
        if batch_idx % 100 == 0:
            print(f"[Epoch {epoch+1}/{epochs}] Batch {batch_idx+1}/{len(trainloader)} Loss: {loss.item():.4f} Accuracy: {acc:.2f}%")

    # Calculate average loss and accuracy for the epoch
    epoch_loss = running_loss / len(trainloader)
    epoch_acc = running_accuracy / len(trainloader)
    train_loss_values.append(epoch_loss)
    train_acc_values.append(epoch_acc)

    # Testing phase
    model.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            test_outputs = model(images)
            test_loss += loss_fn(test_outputs, labels).item()
            test_acc += accuracy_fn(y_true=labels, y_pred=test_outputs.argmax(dim=1))

    # Average test metrics
    avg_test_loss = test_loss / len(testloader)
    avg_test_acc = test_acc / len(testloader)
    test_loss_values.append(avg_test_loss)
    test_acc_values.append(avg_test_acc)

    # Log epoch results
    end_time = time()
    print(f"Epoch {epoch+1} completed in {(end_time - start_time):.2f}s")
    print(f"Train Loss: {epoch_loss:.4f} | Train Accuracy: {epoch_acc:.2f}%")
    print(f"Test Loss: {avg_test_loss:.4f} | Test Accuracy: {avg_test_acc:.2f}%\n")

# Plot training curves
results = {
    "train_loss": train_loss_values,
    "train_acc": train_acc_values,
    "test_loss": test_loss_values,
    "test_acc": test_acc_values
}
plot_loss_curves(results)

In [ ]:
# Start Timer
train_time_start_on_cpu = timer()

# Open a log file
log_file = "training_log.txt"
with open(log_file, "w") as f:
    # Number of Epochs - Change this to train the network for longer
    EPOCHS = 8

    # Training Loop
    for epoch in tqdm(range(EPOCHS)):
        print(f"Epoch: {epoch + 1}")
        train_loss = 0

        # Load the data in batches to avoid memory problems
        for batch, (X, y) in enumerate(train_dataloader):
            X, y = X.to(device), y.to(device)
            # Set model to train
            model.train()
            # Perform the forward pass
            y_pred = model(X)
            # Calculate the loss (how bad our model is at predicting the right values)
            loss = loss_fn(y_pred, y)
            # Accumulate the loss
            train_loss += loss
            # Set the gradients of the optimizer to zero
            optimizer.zero_grad()
            # Perform backpropagation
            loss.backward()
            # Performs a single optimization step
            optimizer.step()
        # Calculate the loss across all the data
        train_loss /= len(train_dataloader)

        # Test the model on unseen data
        test_loss, test_acc = 0, 0
        model.eval()

        with torch.inference_mode():
            for batch, (X_test, y_test) in enumerate(test_dataloader):
                X_test, y_test = X_test.to(device), y_test.to(device)
                # Our model predicts on test data
                test_pred = model(X_test)
                # Calculate loss and accuracy
                test_loss += loss_fn(test_pred, y_test)
                test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))
            # Calculate loss and accuracy
            test_loss /= len(test_dataloader)
            test_acc /= len(test_dataloader)

        # Print and log training and test results for the current epoch
        print(f"Train loss: {train_loss:.4f} | Test loss: {test_loss:.4f} | Accuracy: {test_acc:.2f}%\n")
        f.write(f"Epoch: {epoch + 1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%\n")
        f.flush()

# Stop timer
train_time_end_on_cpu = timer()
# Calculate timer difference
total_train_time_model = print_train_time(start=train_time_start_on_cpu,
                                          end=train_time_end_on_cpu,
                                          device=str(next(model.parameters()).device))

In [ ]:
model.state_dict()

In [ ]:
# Training the Model
# Assuming `model` is already defined under "Creating the Model" section

# Set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training parameters
epochs = 5  # Number of times we pass through the entire dataset

# Training loop
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(trainloader):
        # Move data to the selected device
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if batch_idx % 100 == 99:  # Print every 100 mini-batches
            print(f"Epoch [{epoch + 1}/{epochs}], Batch [{batch_idx + 1}/{len(trainloader)}], Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

print("Training complete.")

In [ ]:
# Import necessary libraries
import torch  # Ensure PyTorch is imported
from torch import nn
from torch.optim import SGD
from tqdm.auto import tqdm
from timeit import default_timer as timer

# Set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training parameters
epochs = 8  # Number of times we pass through the entire dataset

# Open a file to save the training log
log_file_path = "training_log.txt"
with open(log_file_path, "w") as log_file:

    # Start Timer
    train_time_start_on_cpu = timer()

    # Training Loop
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch + 1}")
    train_loss = 0

    # Load the data in batches to avoid memory problems
    for batch, (X, y) in enumerate(trainloader):
        X, y = X.to(device), y.to(device)
        # Set model to train
        model.train()
        # Perform the forward pass
        y_pred = model(X)
        # Calculate the loss (how bad our model is at predicting the right values)
        loss = loss_fn(y_pred, y)
        # Accumulate the loss
        train_loss += loss.item()  # Add the scalar loss value instead of the entire tensor
        # Set the gradients of the optimizer to zero
        optimizer.zero_grad()
        # Perform backpropagation
        loss.backward()
        # Performs a single optimization step
        optimizer.step()

    # Calculate the average train loss across all batches
    train_loss /= len(trainloader)

    # Test the model on unseen data
    test_loss, test_acc = 0, 0
    model.eval()

    # Use inference mode to speed up and prevent gradient calculation
    with torch.inference_mode():
        for batch, (X_test, y_test) in enumerate(testloader):
            X_test, y_test = X_test.to(device), y_test.to(device)
            # Our model predicts on test data
            test_pred = model(X_test)
            # Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y_test).item()  # Get the scalar loss value
            test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

    # Calculate average test loss and accuracy
    test_loss /= len(testloader)
    test_acc /= len(testloader)

    # Print and log training and test results for the current epoch
    log_message = f"Epoch [{epoch + 1}/{epochs}] | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f}\n"
    print(log_message)
    with open(log_file_path, "a") as log_file:
        log_file.write(log_message)

In [ ]:
# Open a log file
log_file = "training_log.txt"
with open(log_file, "w") as f:
    for epoch in range(EPOCHS):
        # Training and evaluation code
        f.write(f"Epoch: {epoch}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%\n")
        f.flush()

## Evaluating the Model

In [ ]:
# Evaluate Model on Unseen Data
def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn):
    """Returns a dictionary containing the results of model predicting on data_loader.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            # Make predictions with the model
            y_pred = model(X)

            # Accumulate the loss and accuracy values per batch
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y,
                                y_pred=y_pred.argmax(dim=1)) # For accuracy, need the prediction labels (logits -> pred_prob -> pred_labels)

        # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)
        acc /= len(data_loader)

    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

# Calculate model 0 results on test dataset
model_results = eval_model(model=model, data_loader=test_dataloader,
    loss_fn=loss_fn, accuracy_fn=accuracy_fn
)
model_results

### Making predictions of random images

In [ ]:
def make_predictions(model: torch.nn.Module, data: list, device: torch.device = device):
  pred_probs = []
  model.eval()
  with torch.inference_mode():
      for sample in data:
          # Prepare sample
          sample = torch.unsqueeze(sample, dim=0).to(device) # Add an extra dimension and send sample to device

          # Forward pass (model outputs raw logit)
          pred_logit = model(sample)

          # Get prediction probability (logit -> prediction probability)
          pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)

          # Get pred_prob off GPU for further calculations
          pred_probs.append(pred_prob.cpu())

  # Stack the pred_probs to turn list into a tensor
  return torch.stack(pred_probs)

In [ ]:
def plot_predictions(test_samples, test_labels):
  # Plot predictions
  plt.figure(figsize=(9, 9))
  nrows = 3
  ncols = 3
  for i, sample in enumerate(test_samples):
    # Create a subplot
    plt.subplot(nrows, ncols, i+1)

    # Plot the target image
    plt.imshow(sample.squeeze(), cmap="gray")

    # Find the prediction label (in text form, e.g. "Sandal")
    pred_label = train_data.classes[pred_classes[i]]

    # Get the truth label (in text form, e.g. "T-shirt")
    truth_label = train_data.classes[test_labels[i]]

    # Create the title text of the plot
    title_text = f"Pred: {pred_label} | Truth: {truth_label}"

    # Check for equality and change title colour accordingly
    if pred_label == truth_label:
        plt.title(title_text, fontsize=10, c="g") # green text if correct
    else:
        plt.title(title_text, fontsize=10, c="r") # red text if wrong
    plt.axis(False);

In [ ]:
import random
test_samples = []
test_labels = []
for sample, label in random.sample(list(test_data), k=9):
    test_samples.append(sample)
    test_labels.append(label)

pred_probs= make_predictions(model=model,
                             data=test_samples)
# Turn the prediction probabilities into prediction labels by taking the argmax()
pred_classes = pred_probs.argmax(dim=1)

# Make Predictions
plot_predictions(test_samples, test_labels)

In [ ]:
wrong_samples = []
wrong_labels = []
counter = 9

while counter > 0:
  randomNum = random.randint(0,len(test_data))
  sample, label = test_data[randomNum]
  model.eval()
  with torch.inference_mode():
    # Prepare sample
    sample = torch.unsqueeze(sample, dim=0).to(device) # Add an extra dimension and send sample to device

    # Forward pass (model outputs raw logit)
    pred_logit = model(sample)

    # Get prediction probability (logit -> prediction probability)
    pred_prob = torch.softmax(pred_logit.squeeze(), dim=0).cpu()
    classNumber = torch.argmax(pred_prob).item()

  if str(classNumber) != str(label):
    wrong_samples.append(sample.cpu())
    wrong_labels.append(label)
    counter -= 1

# Make Predictions
plot_predictions(wrong_samples, wrong_labels)

### Making Confusion Matrix

In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

# 1. Make predictions with trained model
y_preds = []
model.eval()
with torch.inference_mode():
  for X, y in tqdm(test_dataloader, desc="Making predictions"):
    # Send data and targets to target device
    X, y = X.to(device), y.to(device)
    # Do the forward pass
    y_logit = model(X)
    # Turn predictions from logits -> prediction probabilities -> predictions labels
    y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)
    # Put predictions on CPU for evaluation
    y_preds.append(y_pred.cpu())
# Concatenate list of predictions into a tensor
y_pred_tensor = torch.cat(y_preds)

In [ ]:
# See if torchmetrics exists, if not, install it
try:
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")
    assert int(mlxtend.__version__.split(".")[1]) >= 19, "mlxtend verison should be 0.19.0 or higher"
except:
    !pip install -q torchmetrics -U mlxtend # <- Note: If you're using Google Colab, this may require restarting the runtime
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")

In [ ]:
# Import mlxtend upgraded version
import mlxtend
print(mlxtend.__version__)
assert int(mlxtend.__version__.split(".")[1]) >= 19 # should be version 0.19.0 or higher

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# 2. Setup confusion matrix instance and compare predictions to targets
confmat = ConfusionMatrix(num_classes=len(train_data.classes), task='multiclass')
confmat_tensor = confmat(preds=y_pred_tensor,
                         target=test_data.targets)

# 3. Plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(), # matplotlib likes working with NumPy
    class_names=train_data.classes, # turn the row and column labels into class names
    figsize=(10, 7)
);